## FashionMNIST

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Load FashionMNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_loader = DataLoader(
    datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform),
    batch_size=64, shuffle=True
)
test_loader = DataLoader(
    datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform),
    batch_size=1000, shuffle=False
)

In [5]:
# Define a simple CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
# FGSM attack implementation
def fgsm_attack(model, data, target, epsilon):
    data.requires_grad = True
    output = model(data)
    loss = F.cross_entropy(output, target)
    model.zero_grad()
    loss.backward()
    data_grad = data.grad.data
    perturbed_data = data + epsilon * data_grad.sign()
    return perturbed_data


In [7]:
# Adversarial training 
def train(model, device, train_loader, optimizer, epsilon, epoch):
    model.train()
    total_loss, total_adv_loss = 0, 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Standard forward pass
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        total_loss += loss.item()

        # Generate adversarial examples using FGSM
        adv_data = fgsm_attack(model, data, target, epsilon)
        adv_output = model(adv_data)
        adv_loss = F.cross_entropy(adv_output, target)
        total_adv_loss += adv_loss.item()

        # Combined loss
        total_batch_loss = (loss + adv_loss) / 2

        # Backward pass and optimization
        total_batch_loss.backward()
        optimizer.step()

        # Log information for each batch
        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch+1} [{batch_idx * len(data)}/{len(train_loader.dataset)}] '
                  f'Loss: {loss.item():.4f} | Adv Loss: {adv_loss.item():.4f}')

    # Log average loss for epoch
    avg_loss = total_loss / len(train_loader)
    avg_adv_loss = total_adv_loss / len(train_loader)
    print(f'==> Epoch: {epoch+1} | Avg Loss: {avg_loss:.4f} | Avg Adv Loss: {avg_adv_loss:.4f}')


In [10]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"Using device: {device}")

Using device: mps


In [11]:
print(device)

mps


In [12]:
# Training loop
epochs = 5
epsilon = 0.1  
for epoch in range(epochs):
    train(model, device, train_loader, optimizer, epsilon, epoch)
    print(f"Epoch {epoch+1}/{epochs} completed\n")

print("Adversarial training completed.")

Epoch: 1 [0/60000] Loss: 2.3093 | Adv Loss: 2.3551
Epoch: 1 [6400/60000] Loss: 0.5378 | Adv Loss: 0.9141
Epoch: 1 [12800/60000] Loss: 0.4797 | Adv Loss: 0.8660
Epoch: 1 [19200/60000] Loss: 0.5164 | Adv Loss: 0.8877
Epoch: 1 [25600/60000] Loss: 0.2547 | Adv Loss: 0.5873
Epoch: 1 [32000/60000] Loss: 0.5033 | Adv Loss: 0.8968
Epoch: 1 [38400/60000] Loss: 0.3280 | Adv Loss: 0.6940
Epoch: 1 [44800/60000] Loss: 0.4624 | Adv Loss: 0.7919
Epoch: 1 [51200/60000] Loss: 0.2020 | Adv Loss: 0.4735
Epoch: 1 [57600/60000] Loss: 0.3648 | Adv Loss: 0.7154
==> Epoch: 1 | Avg Loss: 0.4065 | Avg Adv Loss: 0.7550
Epoch 1/5 completed

Epoch: 2 [0/60000] Loss: 0.3102 | Adv Loss: 0.5723
Epoch: 2 [6400/60000] Loss: 0.2128 | Adv Loss: 0.4953
Epoch: 2 [12800/60000] Loss: 0.3898 | Adv Loss: 0.7472
Epoch: 2 [19200/60000] Loss: 0.2632 | Adv Loss: 0.5972
Epoch: 2 [25600/60000] Loss: 0.1770 | Adv Loss: 0.4587
Epoch: 2 [32000/60000] Loss: 0.3561 | Adv Loss: 0.6687
Epoch: 2 [38400/60000] Loss: 0.1922 | Adv Loss: 0.5346

Sp, The decrease in both average losses over epochs suggests the model is improving on both clean and adversarial examples, learning to classify both correctly.

## CIFAR-10

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [24]:
# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize for 3 channels
])

train_loader = DataLoader(
    datasets.CIFAR10(root='./data', train=True, download=True, transform=transform),
    batch_size=64, shuffle=True
)
test_loader = DataLoader(
    datasets.CIFAR10(root='./data', train=False, download=True, transform=transform),
    batch_size=1000, shuffle=False
)

Files already downloaded and verified
Files already downloaded and verified


In [25]:
# Define a CNN model for CIFAR-10
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)  # 32x32 -> 30x30
        self.conv2 = nn.Conv2d(32, 64, 3, 1)  # 30x30 -> 28x28
        self.fc1 = nn.Linear(12544, 128)  # Flattened size after conv and pooling
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)  # 28x28 -> 14x14
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [26]:
# FGSM attack implementation
def fgsm_attack(model, data, target, epsilon):
    data.requires_grad = True
    output = model(data)
    loss = F.cross_entropy(output, target)
    model.zero_grad()
    loss.backward()
    data_grad = data.grad.data
    perturbed_data = data + epsilon * data_grad.sign()
    return perturbed_data

In [27]:
# Adversarial training function with logging
def train(model, device, train_loader, optimizer, epsilon, epoch):
    model.train()
    total_loss, total_adv_loss = 0, 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Standard forward pass
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        total_loss += loss.item()

        # Generate adversarial examples using FGSM
        adv_data = fgsm_attack(model, data, target, epsilon)
        adv_output = model(adv_data)
        adv_loss = F.cross_entropy(adv_output, target)
        total_adv_loss += adv_loss.item()

        # Combined loss
        total_batch_loss = (loss + adv_loss) / 2

        # Backward pass and optimization
        total_batch_loss.backward()
        optimizer.step()

        # Log information for each batch
        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch+1} [{batch_idx * len(data)}/{len(train_loader.dataset)}] '
                  f'Loss: {loss.item():.4f} | Adv Loss: {adv_loss.item():.4f}')

    # Log average loss for epoch
    avg_loss = total_loss / len(train_loader)
    avg_adv_loss = total_adv_loss / len(train_loader)
    print(f'==> Epoch: {epoch+1} | Avg Loss: {avg_loss:.4f} | Avg Adv Loss: {avg_adv_loss:.4f}')


In [28]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"Using device: {device}")

Using device: mps


In [29]:
# Training loop
epochs = 5
epsilon = 0.1  # Strength of the adversarial perturbation
for epoch in range(epochs):
    train(model, device, train_loader, optimizer, epsilon, epoch)
    print(f"Epoch {epoch+1}/{epochs} completed\n")

print("Adversarial training completed.")

Epoch: 1 [0/50000] Loss: 2.3125 | Adv Loss: 2.3902
Epoch: 1 [6400/50000] Loss: 1.6778 | Adv Loss: 2.4240
Epoch: 1 [12800/50000] Loss: 1.7606 | Adv Loss: 2.6576
Epoch: 1 [19200/50000] Loss: 1.5561 | Adv Loss: 2.5461
Epoch: 1 [25600/50000] Loss: 1.3406 | Adv Loss: 2.3883
Epoch: 1 [32000/50000] Loss: 1.4365 | Adv Loss: 2.6266
Epoch: 1 [38400/50000] Loss: 1.3080 | Adv Loss: 2.3696
Epoch: 1 [44800/50000] Loss: 1.2193 | Adv Loss: 2.2465
==> Epoch: 1 | Avg Loss: 1.4437 | Avg Adv Loss: 2.4187
Epoch 1/5 completed

Epoch: 2 [0/50000] Loss: 1.3224 | Adv Loss: 2.4332
Epoch: 2 [6400/50000] Loss: 1.2740 | Adv Loss: 2.5509
Epoch: 2 [12800/50000] Loss: 1.2614 | Adv Loss: 2.4981
Epoch: 2 [19200/50000] Loss: 1.0795 | Adv Loss: 2.3307
Epoch: 2 [25600/50000] Loss: 1.0522 | Adv Loss: 2.1670
Epoch: 2 [32000/50000] Loss: 1.2905 | Adv Loss: 2.3692
Epoch: 2 [38400/50000] Loss: 1.1057 | Adv Loss: 2.4354
Epoch: 2 [44800/50000] Loss: 1.1933 | Adv Loss: 2.4381
==> Epoch: 2 | Avg Loss: 1.1330 | Avg Adv Loss: 2.3731